### 作業目的: 實作樹型模型

在本次課程中實作了以Entropy計算訊息增益的決策樹模型，而計算訊息增益的方法除了Entropy只外還有Gini。因此本次作業希望讀者實作以Gini計算

訊息增益，且基於課程的決策樹模型建構隨機森林模型。

在作業資料夾中的`decision_tree_functions.py`檔案有在作業中實作的所有函式，在實作作業中可以充分利用已經寫好的函式

### Q1: 使用Gini計算訊息增益

$$
Gini = \sum_{i=1}^cp(i)(1-p(i)) = 1 - \sum_{i=1}^cp(i)^2
$$

In [18]:
import pandas as pd
import numpy as np
from decision_tree_functions import decision_tree, train_test_split
import random


In [89]:
# 使用與課程中相同的假資料

training_data = [
    ['Green', 3.1, 'Apple'],
    ['Red', 3.2, 'Apple'],
    ['Red', 1.2, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3.3, 'Lemon'],
    ['Yellow', 3.1, 'Lemon'],
    ['Green', 3, 'Apple'],
    ['Red', 1.1, 'Grape'],
    ['Yellow', 3, 'Lemon'],
    ['Red', 1.2, 'Grape'],
]

header = ["color", "diameter", "label"]

df = pd.DataFrame(data=training_data, columns=header)
df.head()

,color,diameter,label
0,Green,3.1,Apple
1,Red,3.2,Apple
2,Red,1.2,Grape
3,Red,1.0,Grape
4,Yellow,3.3,Lemon


In [ ]:
def get_gini(class1, class2):
    return 1 - ((class1/(class1+class2))**2 + (class2/(class1+class2))**2)

In [48]:
#Gini impurity

from collections import defaultdict
def calculate_gini(data):
    
    #取的資料的label訊息
    #print(data,'\n')
    labels = data[:, -1]
    data_len = len(labels)
    #取得所有輸入資料的獨立類別與其個數
    counter = defaultdict(lambda: 1)
    pos_dic = defaultdict(lambda: 1)
    for i in labels:
        counter[i]+=1
    #計算機率
    for i in counter.keys():
        counter[i] = (counter[i]/data_len)**2
    #計算gini impurity
    gini = 1
    for i in counter.keys():
        gini -= counter[i]

    return gini

In [50]:
calculate_gini(df.values)

0.42999999999999994

In [33]:
train_df['label']

0    Apple
1    Apple
2    Grape
3    Grape
4    Lemon
5    Lemon
6    Apple
7    Grape
9    Grape
Name: label, dtype: object

In [142]:
#分割資料集
def train_test_split(df, test_size=0.1):
    if isinstance(test_size, float):
        test_size = round(len(df) * test_size)
    indices = list(df.index)
    test_indices = random.sample(population = indices, k=test_size)
    test_df = df.loc[test_indices]
    train_df = df.drop(test_indices)
    return train_df, test_df
train_df, test_df = train_test_split(df)

#以Gini inpurity作為metric_function訓練決策樹
tree = decision_tree(calculate_gini)
tree0 = tree.fit(train_df)

In [151]:
# 以建構好的樹進行預測
sample = test_df.iloc[0]
print(tree0)
tree.pred(sample,tree0)

{'diameter <= 1.2': ['Grape', {'color = Yellow': ['Lemon', 'Apple']}]}


'Grape'

In [67]:
sample

color         Red
diameter      1.2
label       Grape
Name: 9, dtype: object

### Q2: 實作隨機森林
利用決策樹來實作隨機森林模型，讀者可參考隨機森林課程講義。

此份作業只要求讀者實作隨機sample訓練資料，而隨機sample特徵進行訓練的部分，讀者可以參考`decision_tree_functions.py`中的`get_potential_splits`與`decision_tree`部分(新增參數`random_features`)

In [207]:
from collections import defaultdict
class random_forest():
    '''Random forest model
    Parameters
    ----------
    n_boostrap: int
        number of samples to sample to train indivisual decision tree
    n_tree: int
        number of trees to form a forest
    '''
    
    def __init__(self, n_bootstrap, n_trees, task_type, min_samples, max_depth, metric_function, n_features=None):
        self.n_bootstrap = n_bootstrap
        self.n_trees = n_trees
        self.task_type = task_type
        self.min_samples = min_samples
        self.max_depth = max_depth
        self.metric_function = metric_function
        self.n_features = n_features
    
    def bootstrapping(self, train_df, n_bootstrap):
        #sample data to be used to train individual tree
        print('start bootstrapping')
        print(list(train_df.index), n_bootstrap)
        sample_index = random.sample(population=list(train_df.index), k=n_bootstrap)
        print(sample_index)
        df_bootstrapped = train_df.loc[sample_index]
        print(df_bootstrapped)
        
        #avoid pick the samples with all the same label
        counter = 0
        while df_bootstrapped.iloc[:,-1].nunique() == 1:
            sample_index = random.sample(population=list(train_df.index), k=n_bootstrap)
            df_bootstrapped = train_df.loc[sample_index]
            counter += 1
            if counter == 5:
                print('all label same for five, please check again')
                break
        print('end bootstrapping')
        return df_bootstrapped
    
    def fit(self, train_df):
        print('start fitting')
        self.forest = []
        
        for i in range(self.n_trees):
            df_bootstrapped = self.bootstrapping(train_df, self.n_bootstrap)
            print('call fit for tree')
            tree = decision_tree(calculate_gini)
            tree0 = tree.fit(df_bootstrapped)
            print('tree complete')
            
            self.forest.append(tree0)
        print('finish fitting')
        return self.forest
    
    def pred(self, test_df):
        print('start predicting')
        df_predictions = defaultdict(lambda: 0)
        for tree0 in self.forest:
            tree_ = decision_tree(calculate_gini)
            df_predictions[tree.pred(test_df, tree0)] += 1
        print(df_predictions.items())
        df_predictions = pd.DataFrame.from_dict(df_predictions, orient='index',columns=['label'])
        print(df_predictions)
        
        #majority voting
        random_forest_predictions = df_predictions.nlargest(1,'label')
        print('end predicting')
        return random_forest_predictions.index[0]

In [208]:
train_df, test_df = train_test_split(df, 0.2)

#建立隨機森林模型
forest = random_forest(n_bootstrap=5, n_trees=10, task_type='classification', min_samples=2, max_depth=5, metric_function=calculate_gini, n_features=None)

forest.fit(train_df)

start fitting
start bootstrapping
[0, 2, 3, 4, 5, 6, 7, 9] 5
[5, 7, 6, 2, 0]
    color  diameter  label
5  Yellow       3.1  Lemon
7     Red       1.1  Grape
6   Green       3.0  Apple
2     Red       1.2  Grape
0   Green       3.1  Apple
end bootstrapping
call fit for tree
tree complete
start bootstrapping
[0, 2, 3, 4, 5, 6, 7, 9] 5
[5, 0, 7, 3, 6]
    color  diameter  label
5  Yellow       3.1  Lemon
0   Green       3.1  Apple
7     Red       1.1  Grape
3     Red       1.0  Grape
6   Green       3.0  Apple
end bootstrapping
call fit for tree
tree complete
start bootstrapping
[0, 2, 3, 4, 5, 6, 7, 9] 5
[4, 2, 5, 7, 0]
    color  diameter  label
4  Yellow       3.3  Lemon
2     Red       1.2  Grape
5  Yellow       3.1  Lemon
7     Red       1.1  Grape
0   Green       3.1  Apple
end bootstrapping
call fit for tree
tree complete
start bootstrapping
[0, 2, 3, 4, 5, 6, 7, 9] 5
[3, 7, 2, 9, 4]
    color  diameter  label
3     Red       1.0  Grape
7     Red       1.1  Grape
2     Red       1

[{'diameter <= 1.2': ['Grape', {'color = Yellow': ['Lemon', 'Apple']}]},
 {'diameter <= 1.1': ['Grape', {'color = Yellow': ['Lemon', 'Apple']}]},
 {'diameter <= 1.2': ['Grape', {'color = Yellow': ['Lemon', 'Apple']}]},
 {'diameter <= 1.2': ['Grape', 'Lemon']},
 {'diameter <= 1.1': ['Grape', {'color = Yellow': ['Lemon', 'Apple']}]},
 {'diameter <= 1.1': ['Grape', {'color = Yellow': ['Lemon', 'Apple']}]},
 {'diameter <= 1.2': ['Grape', {'color = Yellow': ['Lemon', 'Apple']}]},
 {'diameter <= 1.2': ['Grape', 'Lemon']},
 {'diameter <= 1.2': ['Grape', {'color = Yellow': ['Lemon', 'Apple']}]},
 {'diameter <= 1.2': ['Grape', {'diameter <= 3.1': ['Apple', 'Lemon']}]}]

In [210]:
#{'diameter <= 1.2': ['Grape', {'color = Yellow': ['Lemon', 'Apple']}]}
forest.pred(test_df.iloc[1])

start predicting
dict_items([('Apple', 7), ('Lemon', 3)])
       label
Apple      7
Lemon      3
end predicting


'Apple'

In [212]:
test_df.iloc[1]

color         Red
diameter      3.2
label       Apple
Name: 1, dtype: object